### <center> Testes de Implementação: **Ollama e Langchain** em Agentes Inteligentes <center>


Importações

In [ ]:
from pydantic import BaseModel, Field
from langchain.tools import BaseTool
from typing import Optional, Type
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.tools.render import render_text_description_and_args
import re
import ollama
import chromadb
from langchain.prompts import PromptTemplate

True

Extratação do Documento

In [2]:
faq = ""

with open('faq.txt', 'r', encoding='utf-8') as file:
        faq = file.read()

documents = re.split(r'(?=Q: )', faq)

for i in reversed(documents):
    if i == "":
        documents.remove(i)

print(documents)

['**', 'Q: "How do I enable automatic updates?"**  \nA: "*Android*  \nSEMPRE JBS was developed to provide faster information and convenience in your daily life. Therefore, when updates are available, you don’t need to worry about checking for the new version. The app will notify you about the new version and direct you to the Google Play Store. It’s easy and quick. Don’t worry 😊  \n\n*iOS*  \nIf you want your phone to automatically update to the latest versions, simply:  \n1. Go to “Settings”;  \n2. Click on “Apple Store”;  \n3. Select the “Automatic Updates” option."  \n\n---\n\n**', 'Q: "How do I install Sempre JBS?"**  \nA: "*Android*  \nHere is the link to install on Android: [https://play.google.com/store/apps/details?id=br.com.jbs.rh.comvoce&hl=pt_BR&gl=US](https://play.google.com/store/apps/details?id=br.com.jbs.rh.comvoce&hl=pt_BR&gl=US)  \n\n*iOS*  \nHere is the link to install on iOS: [botm.cc/l/8eQNDoj](botm.cc/l/8eQNDoj)"  \n\n---\n\n**', 'Q: "I want to update my app. How d

Conexão com o ChromaDB

In [3]:
client = chromadb.Client()
collection = client.create_collection(name="docs")

# Guardar os documentos em um vetor para embedding 
for i, d in enumerate(documents):
    response = ollama.embed(model="mxbai-embed-large", input=d)
    embeddings = response["embeddings"]
    collection.add(
        ids=[str(i)],
        embeddings=embeddings,
        documents=[d]
    )

Configuração de Tools

In [4]:
class EmbeddingInput(BaseModel):
    question: str = Field(..., description="The user's question requiring technical information or data.")

class EmbeddingTool(BaseTool):
    name: str = "Embedding"
    description: str = (
        "Useful when the user needs technical information or any question requiring data to provide an answer."
    )
    args_schema: Type[BaseModel] = EmbeddingInput

    def _get_embedding(self, text: str):
        """Generates embedding for the given text."""
        try:
            response = ollama.embed(model="mxbai-embed-large", input=text)
            return response["embeddings"][0]
        except Exception as e:
            raise ValueError(f"Error generating embedding: {str(e)}")

    def _query_collection(self, embedding, num_results=1):
        """Queries ChromaDB using the given embedding."""
        try:
            results = collection.query(
                query_embeddings=[embedding],
                n_results=num_results
            )
            return results
        except Exception as e:
            raise ValueError(f"Error querying ChromaDB: {str(e)}")

    def _run(
        self,
        question: Optional[str] = None,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Gets relevant information for the question."""
        if not question:
            return "No question was provided."

        try:
            embedding = self._get_embedding(question)

            results = self._query_collection(embedding, num_results=1)

            if results and results["documents"]:
                return (
                    f"Use this information to answer the question. "
                    f"If no relevant information is found, inform the user that it wasn't possible to answer. "
                    f"Information: {results['documents'][0][0]}"
                )
            else:
                return "No relevant information was found to answer the question."
        
        except ValueError as e:
            return str(e)
        except Exception as e:
            return f"Error processing the question: {str(e)}"


In [5]:
response = (
    "Create a final answer that says if they "
    "have any questions about movies or actors"
)


class SmalltalkInput(BaseModel):
    query: Optional[str] = Field(description="user query")


class SmalltalkTool(BaseTool):
    name: str = "Smalltalk"
    description: str = "useful for when user greets you or wants to smalltalk" 
    args_schema: Type[BaseModel] = SmalltalkInput

    def _run(
        self,
        query: Optional[str] = None,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool."""
        return response

    async def _arun(
        self,
        query: Optional[str] = None,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        return response

In [6]:
tools = [EmbeddingTool(), SmalltalkTool()]

System Prompts e Question

In [7]:
system_prompt_tool_call = f"""
You are a tool calling agent, your is choose a tool tho solve the user's doubt.

You have access to the following tools: 
{render_text_description_and_args(tools).replace('{', '{{').replace('}', '}}')}

You should choose a tool that most covens to the situation.
Do not worry, you can have access to all the information you are asked about.
"""

system_prompt_model = PromptTemplate.from_template(
"""
Contexto: Você é um assistente virtual treinado para ajudar os colaboradores da empresa JBS a esclarecer dúvidas ou resolver problemas sobre os valores da empresa ou o aplicativo de RH chamado Sempre JBS.
Restrito aos dados fornecidos: Responda exclusivamente com base nas informações apresentadas nos exemplos. Não inclua suposições, interpretações ou dados adicionais que não estejam explicitamente descritos nos exemplos fornecidos.
Conciso e direto: Responda de forma objetiva e clara.
Idioma: Sempre em PT-BR.
Tratamento de casos específicos:
- Informações não encontradas ou perguntas fora do contexto: "Não localizei essa informação no documento disponível. Por favor, entre em contato com o RH para mais detalhes."
- Perguntas irrelevantes: "Esta pergunta não se refere aos valores da JBS ou ao aplicativo Sempre JBS. Portanto, não posso respondê-la."
- Perguntas ambíguas: "Por favor, forneça mais detalhes para que eu possa responder com precisão."
Tom da resposta: Formal e educado.

Encontre as informações necessárias para a formulação da resposta nesses exemplos disponíveis: {tool_response}

Responda à pergunta em PT-BR:
"""
)

question = "how to download the app?"

JSON Tools

In [8]:
json_tools = [
        {
            'type': 'function',
            'function': {
                'name': 'Embedding',
                'description': 'Useful when the user needs technical information or any question requiring data to provide an answer.',
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'question': {
                            'type': 'string',
                            'description': 'The user\'s question requiring technical information or data.'
                        }
                    },
                    'required': ['question']
                },
            },
        },
        {
            'type': 'function',
            'function': {
                'name': 'Smalltalk',
                'description': 'Useful for when the user greets you or wants to engage in small talk about HR-related topics.',
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'query': {
                            'type': 'string',
                            'description': 'The user query, greeting, or small talk input.'
                        }
                    },
                    'required': ['query']
                },
            },
        }
    ]

Funções de Execução

In [9]:
def get_tool_call(system_prompt, question, json_tools):
    tool_call = ollama.chat(
        model = "llama3.1",
        messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": question}],
        tools = json_tools
    )
    
    return tool_call

def get_tool_response(function_name, function_arguments):    
    if function_name == "Embedding":
        tool_response = EmbeddingTool()._run(question=function_arguments['question'])
    elif function_name == "SmallTalk":
        tool_response = SmalltalkTool()._run(query=function_arguments['question'])
    else:
        tool_response = None

    return tool_response

def get_model_response(tool_response, system_prompt_model, question):
    system_prompt_model_formatted = system_prompt_model.format(tool_response=tool_response)
    model_response = ollama.chat(
        model = "llama3.1",
        messages = [{"role": "system", "content": system_prompt_model_formatted}, {"role": "user", "content": question}]
    )

    return model_response

In [10]:
tool_call = get_tool_call(system_prompt_tool_call, question, json_tools)

tool_calls = tool_call.message.tool_calls
for tool_call in tool_calls:
    function_name = tool_call.function.name
    arguments = tool_call.function.arguments

In [11]:
tool_response = get_tool_response(function_name, arguments)

In [12]:
model_response = get_model_response(tool_response, system_prompt_model, question)
model_response.message.content

'Não localizei essa informação no documento disponível. Por favor, entre em contato com o RH para mais detalhes.'